## Step 40: Build SWC Stations Dataset

Purpose: Combine COOP and ISD stations into a single SWC station dataset.

Note: this notebook is relatively short as its expected additional station processing 
logic will be added at a later date.

In [3]:
import arcpy;
import os,sys;
import importlib;

print("Step 40: Build SWC Stations Dataset");

import swc_resources;
importlib.reload(swc_resources);
rez = swc_resources.rez();

coop = rez['source'] + os.sep + 'COOP_STATIONS_TO_USE';
coop_cnt = arcpy.GetCount_management(coop)[0];
isd  = rez['source'] + os.sep + 'ISD_STATIONS_TO_USE';
isd_cnt = arcpy.GetCount_management(isd)[0];

print("  COOP Stations    : " + str(coop_cnt));
print("  ISD Stations     : " + str(isd_cnt));


Step 40: Build SWC Stations Dataset
  COOP Stations    : 1851
  ISD Stations     : 3293


### 40.010: Create fresh SWC_Station_Universe working dataset

Note any additional steps to filter or adjust the spatial aspect of the station universe should occur between
steps 010 and 030.

In [12]:
%%time

output40010 = rez['results'] + os.sep + 'SWC_Station_Universe';

print("  creating new station universe feature class");
if arcpy.Exists(output40010):
    arcpy.Delete_management(output40010);
  
arcpy.CreateFeatureclass_management(
     out_path      = os.path.dirname(output40010)
    ,out_name      = os.path.basename(output40010)
    ,geometry_type = "POINT"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269) 
);

arcpy.management.AddFields(
     in_table          = output40010
    ,field_description = [
         ['StationId'       ,'TEXT'  ,'Station ID'        ,14 ]
        ,['DataType'        ,'TEXT'  ,'Data Type'         ,255]
        ,['FileName'        ,'TEXT'  ,'FileName'          ,255]
        ,['Id'              ,'LONG'  ,'Id'                    ]
        ,['Lat'             ,'DOUBLE','Lat'                   ]
        ,['Long'            ,'DOUBLE','Long'                  ]
        ,['Scenario'        ,'TEXT'  ,'Scenario'          ,255]
        ,['Constituent'     ,'TEXT'  ,'Constituent'       ,255]
        ,['SDate'           ,'TEXT'  ,'Start Date'        ,12 ]
        ,['EDate'           ,'TEXT'  ,'End Date'          ,12 ]
        ,['YrCount'         ,'DOUBLE','Year Count'            ]
        ,['Value'           ,'DOUBLE','Value'                 ]
        ,['StaNam'          ,'TEXT'  ,'Station Name'      ,255]
        ,['StationSource'   ,'TEXT'  ,'Station Source'    ,255]
        ,['StationStateCode','TEXT'  ,'Station State Code',2  ]
        ,['CONUSFlag'       ,'TEXT'  ,'CONUS Flag'        ,1  ]
    ]
);

print("  adding indexes");
z = arcpy.management.AddIndex(
     in_table   = output40010
    ,fields     = 'StationId'
    ,index_name = 'StationId_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output40010
    ,fields     = 'StationSource'
    ,index_name = 'StationSource_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output40010
    ,fields     = 'StationStateCode'
    ,index_name = 'StationStateCode_IDX'
);

z = arcpy.management.AddIndex(
     in_table   = output40010
    ,fields     = 'CONUSFlag'
    ,index_name = 'CONUSFlag_IDX'
);


  creating new station universe feature class
  adding indexes
Wall time: 3.66 s


### 40.020: Load COOP and ISD station data into SWC_Station_Universe feature class

In [13]:
%%time

output40020 = rez['results'] + os.sep + 'SWC_Station_Universe';
coop        = rez['source']  + os.sep + 'COOP_STATIONS_TO_USE';
isd         = rez['source']  + os.sep + 'ISD_STATIONS_TO_USE';

fldout = [
     'StationId'
    ,'DataType'
    ,'FileName'
    ,'Id'
    ,'Lat'
    ,'Long'
    ,'Scenario'
    ,'Constituent'
    ,'SDate'
    ,'EDate'
    ,'YrCount'
    ,'Value'
    ,'StaNam'
    ,'StationSource'
    ,'StationStateCode'
    ,'CONUSFlag'
    ,'SHAPE@'
];

fldcoop = [
     'station_id'
    ,'station_name'
    ,'state'
    ,'start_date_clean'
    ,'end_date_clean'
    ,'latitude'
    ,'longitude'
    ,'in_basins'
    ,'break_with_basins'
    ,'network'
    ,'year_count'
    ,'start_date_to_use_clean'
    ,'end_date_to_use_clean'
    ,'SHAPE@'
];

fldisd = [
     'station_id'
    ,'station_name'
    ,'state'
    ,'start_date_clean'
    ,'end_date_clean'
    ,'latitude'
    ,'longitude'
    ,'in_basins'
    ,'break_with_basins'
    ,'network'
    ,'year_count'
    ,'SHAPE@'
];

with arcpy.da.InsertCursor(
     in_table    = output40020
    ,field_names = fldout
) as outcur:

    print("  loading COOP stations");
    with arcpy.da.SearchCursor(
         in_table    = coop
        ,field_names = fldcoop
    ) as incur:
        for row in incur:
            
            state_code = row[2].upper();
            
            conus_flag = 'Y';
            if state_code in ['AS','GU','MP','HI','AK','PR','VI']:
                conus_flag = 'N'
            
            outcur.insertRow((
                 row[0]
                ,'WdmFinal'
                ,state_code.lower() + str(row[0]) + '.txt'
                ,1
                ,row[5]
                ,row[6]
                ,'OBSERVED'
                ,'PREC'
                ,row[11]
                ,row[12]
                ,row[10]
                ,0.0
                ,row[1]
                ,'COOP'
                ,state_code
                ,conus_flag
                ,row[13]
            ));
            
    print("  loading ISD stations");
    with arcpy.da.SearchCursor(
         in_table    = isd
        ,field_names = fldisd
    ) as incur:
        for row in incur:
            
            state_code = row[2].upper();
            
            conus_flag = 'Y';
            if state_code in ['AS','GU','MP','HI','AK','PR','VI']:
                conus_flag = 'N'
                
            outcur.insertRow((
                 row[0]
                ,'WdmFinal'
                ,state_code.lower() + str(row[0]) + '.txt'
                ,1
                ,row[5]
                ,row[6]
                ,'OBSERVED'
                ,'PREC'
                ,row[3]
                ,row[4]
                ,row[10]
                ,0.0
                ,row[1]
                ,'ISD'
                ,state_code
                ,conus_flag
                ,row[11]
            ));

output40020_cnt = arcpy.GetCount_management(output40020)[0];
print("  working station count: " + str(output40020_cnt));


  loading COOP stations
  loading ISD stations
  working station count: 5144
Wall time: 1.7 s


### 40.030: Review and QA

QA Products:

1. station counts by state saved to **step40qa.txt**


In [16]:
%%time

input40030 = rez['results'] + os.sep + 'SWC_Station_Universe';
stat_cnt   = arcpy.GetCount_management(input40030)[0];

print("  New Stations: " + str(stat_cnt));

if arcpy.Exists('memory/stats'):
    arcpy.Delete_management('memory/stats');
    
arcpy.conversion.FeatureClassToFeatureClass(
     in_features  = input40030
    ,out_path     = 'memory'
    ,out_name     = 'stats'
);
    
if arcpy.Exists('memory/stats2'):
    arcpy.Delete_management('memory/stats2');
 
arcpy.Statistics_analysis(
     in_table          = 'memory/stats'
    ,out_table         = 'memory/stats2'
    ,statistics_fields = [["FileName","COUNT"]]
    ,case_field        = ['StationStateCode']
);

with open(rez['qa'] + os.sep + 'step40qa.txt',"w") as out:
    out.write("Step 40 QA Review\n");
    out.write(datetime.datetime.now().isoformat() + "\n");
    out.write("Total Stations," + str(stat_cnt) + "\n");

    with arcpy.da.SearchCursor(
         in_table    = 'memory/stats2'
        ,field_names = ['StationStateCode','COUNT_FileName']
        ,sql_clause  = (None,'ORDER BY StationStateCode')
    ) as cur:
        
        for row in cur:
            out.write(row[0] + ',' + str(row[1]) + "\n");


  New Stations: 5144
Wall time: 999 ms
